In [ ]:
#Use the following in a bash script
# apt install tesseract-ocr libtesseract-dev poppler-utils

In [18]:
pip install -r ../requirements.in

Note: you may need to restart the kernel to use updated packages.


In [19]:
from unstructured.partition.pdf import partition_pdf
from unstructured.documents.elements import Title, NarrativeText, Text
from unstructured.chunking.basic import chunk_elements
from typing import List
import os
import weaviate
from weaviate.util import generate_uuid5
import ollama
from pprint import pprint
import json

In [ ]:
ollama.pull("mxbai-embed-large:v1")


In [27]:
ollama.list()

{'models': [{'name': 'mxbai-embed-large:v1',
   'model': 'mxbai-embed-large:v1',
   'modified_at': '2024-04-20T21:03:01.270699797Z',
   'size': 669615493,
   'digest': '468836162de7f81e041c43663fedbbba921dcea9b9fefea135685a39b2d83dd8',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'bert',
    'families': ['bert'],
    'parameter_size': '334M',
    'quantization_level': 'F16'}}]}

In [ ]:
# Use this for now
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH)

In [ ]:
#save this for later
FILE_PATH = "../docs/Player_s Handbook.pdf"

elements = partition_pdf(filename=FILE_PATH, strategy="hi_res", infer_table_structure=True)

In [20]:
import weaviate
import weaviate.classes as wvc
client = weaviate.connect_to_local(
    additional_config=wvc.init.AdditionalConfig(
        timeout=(60,1800),
    ),
)
client.connect()

In [ ]:
import weaviate.classes as wvc
from weaviate.collections import Collection
from weaviate.client import WeaviateClient

def create_collection(client: WeaviateClient, collection_name: str)-> Collection:
    with client: 
        client.collections.delete(collection_name)
        client.collections.create(
            name=collection_name,
            vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_transformers(),
            vector_index_config=wvc.config.Configure.VectorIndex.hnsw(
                distance_metric=wvc.config.VectorDistances.COSINE # select prefered distance metric
        ),
    )
    collection = client.collections.get(name=collection_name)
    return collection

collection = create_collection(client, 'PHB')


In [ ]:
narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]
titles = [elem for elem in elements if elem.category == "Title"]
uncat_texts = [elem for elem in elements if elem.category == "UncategorizedText"]


In [ ]:

for text in narrative_texts:
    pprint(text.to_dict())

In [ ]:
for title in titles:
    pprint(title.to_dict())

In [ ]:
for text in uncat_texts:
    pprint(text.to_dict())

In [ ]:
for elem in elements:
    pprint(elem.to_dict())

In [ ]:
element_dicts = [chunk.to_dict() for chunk in elements]



In [12]:
embeddings = []

for element in element_dicts:
    response = ollama.embeddings(model='mxbai-embed-large:v1', prompt=element['text'])
    embedding = response["embedding"]
    embeddings.append(embedding)
    

In [13]:
embeddings

NameError: name 'embeddings' is not defined

In [13]:
chunk_embeddings_with_metadata = [
        {
            "id":  None,
            "type": element['type'],
            "title": element['metadata']['filename'],
            "url": "None",
            "content": element['text'],
            "label": "No Label",
            "tokens": len(element['text'].split()),
            "embedding": embedding,
        }
        for element, embedding in zip(element_dicts, embeddings)
    ]

In [20]:
from typing import List, Dict 
from weaviate.client import WeaviateClient
import weaviate.classes as wvc
def load_chunks_into_weaviate(chunks: List[Dict], client: WeaviateClient, collection_name: str):
    collection = create_collection(client, collection_name)
    chunk_objs = []
    for chunk in chunks:
        chunk_obj = wvc.data.DataObject(
            properties={
                "content": chunk['content'],
                "tokens": chunk['tokens'],
                "title": chunk['title'],
                "type": chunk['type'],
                "url": chunk['url'],
                "label": chunk['label']
            }
        )    
        chunk_objs.append(chunk_obj)
        
    with client:
        collection.data.insert_many(chunk_objs)
        
    print(f"Loaded {len(chunks)} chunks into Weaviate")
        
load_chunks_into_weaviate(chunk_embeddings_with_metadata, client, 'PHB')

Loaded 13847 chunks into Weaviate


In [21]:
import ollama
prompt = "You are a Dungeons & Dragons assistant. You provide answers to questions in a helpful manner."
test_query = "What deity would a good-aligned paladin worship?"

test_response = ollama.embeddings(
    prompt=prompt,
    model='mxbai-embed-large:v1'
)

In [24]:
import weaviate.classes as wvc
from weaviate.collections import Collection
from weaviate.client import WeaviateClient

def get_collection(client: WeaviateClient, collection_name: str)-> Collection:
    collection = client.collections.get(collection_name)
    return collection

client.connect()
test_collection = get_collection(client, "PHB")
print(test_collection)


<weaviate.Collection config={
  "name": "PHB",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false
  },
  "properties": [
    {
      "name": "type",
      "description": "This property was generated by Weaviate's auto-schema feature on Sun Apr 21 12:57:43 2024",
      "data_type": "text",
      "index_filterable": true,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": {
        "skip": false,
        "vectorize_property_name": false
      },
      "vectorizer": "text2vec-transformers"
    },
    {
      "name": "tokens",
      "description": "This proper

In [26]:

test_results = test_collection.query(
    query_embeddings = [test_response["embedding"]],
    n_results = 5
)

TypeError: '_QueryCollection' object is not callable

In [ ]:
ollama.pull("mistral")

test_output = ollama.generate(
    model="mistral"
    prompt = f"Using this data: {test_response}, respond to this prompt: {test_query}"
)